In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

from collections import Counter
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import defaultdict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_json('/content/drive/MyDrive/resume/dataset.json',lines=True)

In [ ]:
filtered_df = df[df['category'].isin(['COMEDY','BUSINESS', 'SPORTS', 'CRIME','RELIGION','HEALTHY LIVING', 'POLITICS'])]
filtered_df_shuffle = filtered_df.sample(frac=1,random_state=42)

total_rows = len(filtered_df_shuffle)
train_rows = int(0.85*total_rows)
test_rows = total_rows-train_rows

train_df = filtered_df_shuffle.iloc[:train_rows, :]
test_df = filtered_df_shuffle.iloc[train_rows:, :]

In [ ]:
uniq_categ = train_df['category'].value_counts()
total =0
for i in range(0,len(uniq_categ)):
  total+=uniq_categ[i]

total

48682

In [ ]:
uniq_cat = test_df['category'].value_counts()
total_test =0
for i in range(0,len(uniq_cat)):
  total_test+=uniq_cat[i]

total_test

8592

In [ ]:
prior = {}
for category,count in uniq_categ.items():
  prob = count/total
  prior[category] = {'count':count,'prob': prob}

for category in uniq_categ.items():
  print(category[0])

POLITICS
HEALTHY LIVING
BUSINESS
SPORTS
COMEDY
CRIME
RELIGION


In [ ]:
prior_test = {}
for category,count in uniq_cat.items():
  prob = count/total_test
  prior_test[category] = {'count':count,'prob': prob}

for category in uniq_cat.items():
  print(category[0])

POLITICS
HEALTHY LIVING
COMEDY
BUSINESS
SPORTS
CRIME
RELIGION


In [ ]:
all_desc = ' '.join(train_df['short_description'])
all_words = re.findall(r'\b\w+\b',all_desc.lower())
categ_word_count = {}
unique_words_count = {}
unique_words_category = {}
filtered= []
import string
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

for category in train_df['category'].unique():
  category_df = train_df[train_df['category']==category]
  category_word = []
  for i in range(0,len(category_df)):
    string = category_df.iloc[i]['short_description']
    string1 = re.findall(r'\b\w+\b',string.lower())
    string2 = word_tokenize(string.lower())
    string3 = set(string2)
    string4 = list(string3)
    content_word = [word for word in string4 if word.isalpha() and word not in stop_words]
    category_word += content_word
  unique_words = set(category_word)
  unique_words_category[category]=unique_words
  filtered = category_word
  word_freq = FreqDist(category_word)
  categ_word_count[category] = word_freq
  unique_words_count[category] = len(unique_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(prior['POLITICS']['count'])
print(prior_test['POLITICS']['count'])

27849
4890


In [ ]:
import numpy as np
correct =0
category_count={}
category_correct={}
for category in uniq_categ.items():
  category_count[category[0]] = 0  # Initialize the count for each category
  category_correct[category[0]] = 0  # Initialize the correct prediction count for each category
for i in range(0,len(test_df)):
  #print(test_df.iloc[i])
  string = test_df.iloc[i]['short_description']
  cat = test_df.iloc[i]['category']
  words1 = re.findall(r'\b\w+\b',string.lower())
  words = word_tokenize(string.lower())
  word = set(words)
  string4 = list(word)
  content_word = [word for word in string4 if word.isalpha() and word not in stop_words]
  res_prob={}
  for category in uniq_categ.items():
    group_prob=np.log(prior[category[0]]['prob'])
    for w in content_word:
      word_count = categ_word_count[category[0]][w]
      res = np.log((1+word_count))-np.log((2+prior[category[0]]['count']))
      group_prob+=res

    res_prob[category[0]]= group_prob

  #print(res_prob)
  max_categ = max(res_prob,key=res_prob.get)
  category_count[category[0]]+=1
  # print(max_categ)
  if cat==max_categ:
    category_correct[max_categ]+=1
    correct+=1
  #print("predicted: ",max_categ)
  #print("---------------------------------------")
for category in uniq_categ.items():
  print(category[0]," = ",((category_correct[category[0]])/(prior_test[category[0]]['count']))*100)
print(((correct)/len(test_df))*100)




POLITICS  =  62.78118609406953
HEALTHY LIVING  =  47.72065955383123
BUSINESS  =  59.45945945945946
SPORTS  =  34.455128205128204
COMEDY  =  9.404388714733543
CRIME  =  48.33759590792839
RELIGION  =  52.44215938303341
53.58472998137802
